In [6]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Local/dcgan"

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Local/dcgan
Found GPU at: /device:GPU:0


Comment out the above if not using Google Colaboratory. 

## Define the Network
Below is the code used to create the DCGAN. Note that much of this code is taken from https://github.com/eriklindernoren/Keras-GAN. The main modifications here are some simplifications to variable names, added functionality for saving images, and the ability to plot the progression of the GAN over time on the same noise batch. 

In [0]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import imageio

In [0]:
class DCGAN(): 
  def __init__(self, name= "mnist", imgrows=28, imgcols=28, channels= 1, latent_dim= 100):
    self.name= name
    self.img_rows= imgrows
    self.img_cols= imgcols
    self.channels= channels
    self.img_shape= (self.img_rows, self.img_cols, self.channels)
    self.latent_dim= latent_dim
    
    #parameters chosen from the paper by Radford et al.
    optimizer= Adam(0.0002, 0.5)
    
    #construct generator
    self.generator= self.generator()
    
    #construct discriminator
    self.discriminator= self.discriminator()
    self.discriminator.compile(loss= "binary_crossentropy",
                              optimizer= optimizer, 
                              metrics= ['accuracy'])
    
    #generator generates images from noise
    noise= Input(shape= (self.latent_dim, ))
    img= self.generator(noise)
    
    #only train generator
    self.discriminator.trainable= False
    
    #validity of image as determined by discriminator
    val= self.discriminator(img)
    
    #stack generator and discriminator
    self.GAN = Model(noise, val)
    self.GAN.compile(loss= "binary_crossentropy", optimizer= optimizer)
    
  
  def generator(self): 
    model = Sequential()
    model.add(Dense(128 * 7 * 7, activation= "relu", input_dim= self.latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size= 3, padding= "same"))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size= 3, padding= "same"))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(self.channels, kernel_size= 3, padding= "same"))
    model.add(Activation("tanh"))
    print("Generator Summary ... ")
    model.summary()
    
    #input noise into generator
    noise= Input(shape= (self.latent_dim, ))
    #pass noise through model to retrieve image
    img= model(noise)
    
    return Model(noise, img)
  
  def discriminator(self): 
    model= Sequential()
    model.add(Conv2D(32, kernel_size= 3, strides= 2, input_shape= self.img_shape, padding= "same"))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size= 3, strides= 2, padding= "same"))
    model.add(ZeroPadding2D(padding= ((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size= 3, strides= 2, padding= "same"))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides= 1, padding= "same"))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation= "sigmoid"))
    print("Discriminator Summary ... ")
    model.summary()
    
    img= Input(shape= self.img_shape)
    val= model(img)
    return Model(img, val)
  
  def images(self, noise, rows= 5, cols= 5, name= "untitled", title= "",
             show= False): 
    generated= self.generator.predict(noise)
    
    #rescale images to [0, 1]
    generated= 0.5* generated + 0.5
    
    fig, ax= plt.subplots(rows, cols)
    count= 0
    for row in range(rows): 
      for col in range(cols): 
        ax[row, col].imshow(generated[count, :, :, 0], cmap= "gray")
        ax[row, col].axis("off")
        count += 1
    
    fig.suptitle(title)    
    fig.savefig("%s.png" % name)
    
    # Used to return the plot as an image rray
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    if not show: 
      plt.close()
    
    return image
    
      
  def train(self, images, epochs= 8000, batch_size= 32, 
           interval= 100, rows= 5, cols= 5):
    #purely for saving purposes ...
    fixed_noise_batch= np.random.normal(0, 1, (rows * cols, self.latent_dim))
    plots= []
    
    #if the directory specified by name doesn't exist, create it
    if not os.path.exists(self.name):
       os.makedirs(self.name)
    
    images= images / 127.5 - 1
    images= np.expand_dims(images, axis= 3)

    #adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs + 1):
      #train discriminator....
      #select random images
      imgs= images[np.random.randint(0, images.shape[0], batch_size)]

      #create random noise and generate new images
      noise= np.random.normal(0, 1, (batch_size, self.latent_dim))
      generated= self.generator.predict(noise)

      d_loss_real= self.discriminator.train_on_batch(imgs, valid)
      d_loss_fake= self.discriminator.train_on_batch(generated, fake)
      d_loss= 0.5* np.add(d_loss_real, d_loss_fake)
      
      #train generator, by creating images that seem like valid ones
      g_loss= self.GAN.train_on_batch(noise, valid)
      
      if epoch % interval == 0:
        status= "[%d] D loss: %f, acc: %.2f%% || G loss: %f" %(epoch, 
                 d_loss[0], d_loss[1] * 100, g_loss)
        print(status)
        
        #generate images
        name= "%s/%d" % (self.name, epoch)
        plot= self.images(name= name, noise= fixed_noise_batch, 
                          title= status, rows= rows, cols= cols)
        plots.append(plot)
    
    #save a gif of the progression for visualization
    imageio.mimsave("%s/progression.gif" % self.name, np.array(plots), fps=8)
    

## Run the Code
Below is the code to run the dcgan on the mnist dataset. 

In [9]:
(X_train, _), (_, _) = mnist.load_data()
dcgan= DCGAN()

Generator Summary ... 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_2 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization_6 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_4 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 28, 28, 128)     

In [10]:
dcgan.train(images= X_train)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[0] D loss: 1.096302, acc: 31.25% || G loss: 0.630800
[100] D loss: 0.706408, acc: 60.94% || G loss: 1.358278
[200] D loss: 0.926362, acc: 31.25% || G loss: 0.880062
[300] D loss: 0.866403, acc: 45.31% || G loss: 1.068562
[400] D loss: 0.819774, acc: 35.94% || G loss: 0.900864
[500] D loss: 0.768051, acc: 51.56% || G loss: 1.025126
[600] D loss: 0.735108, acc: 62.50% || G loss: 1.146107
[700] D loss: 0.703209, acc: 60.94% || G loss: 0.998821
[800] D loss: 0.694913, acc: 59.38% || G loss: 1.007398
[900] D loss: 0.669461, acc: 65.62% || G loss: 1.033973
[1000] D loss: 0.761393, acc: 48.44% || G loss: 0.991288
[1100] D loss: 0.671172, acc: 64.06% || G loss: 0.823637
[1200] D loss: 0.667428, acc: 60.94% || G loss: 1.013246
[1300] D loss: 0.859505, acc: 42.19% || G loss: 0.843229
[1400] D loss: 0.700311, acc: 56.25% || G loss: 0.967336
[1500] D loss: 0.650069, acc: 59.38% || G loss: 1.006164
[1600] D loss: 0.737905, acc: 46.88% || G loss: 0.955496
[1700] D loss: 0.693388, acc: 56.25% || G l